# 以下の論文を読み問題に答えてください。  
# CNNを使った物体検出（Object Detection）の代表的な研究です。

[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99  
https://arxiv.org/pdf/1506.01497.pdf  
(1) 物体検出の分野にはどういった手法が存在したか。  
(2) Fasterとあるが、どういった仕組みで高速化したのか。  
(3) One-Stageの手法とTwo-Stageの手法はどう違うのか。  
(4) RPNとは何か。  
(5) RoIプーリングとは何か。  
(6) Anchorのサイズはどうするのが適切か。  
(7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。  
(8) （アドバンス）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。  
答える際は論文のどの部分からそれが分かるかを書く。  
必要に応じて先行研究（引用されている論文）も探しにいく。最低2つは他の論文を利用して回答すること。  
論文の紹介記事を見ても良い。ただし、答えは論文内に根拠を探すこと。  

# (1) 物体検出の分野にはどういった手法が存在したか。

SPPnet,Fast-CNNなどが存在した。

物体があるであろう領域をregion-of-interest(ROI)などと呼ぶ。またROIを提案するモデルをregion proposalと呼ぶ。  
しかし、ROIを特定することなく、画像を細かく分割し、分割した数千のエリア全てをCNNに突っ込むという力技を行なっていた。  
すると、あるクラスに関して確率が高いと判定された部分に物体が存在し、逆にクラスの判定結果がランダムに近い状態だと背景だという検出を行なった。  
しかし、１枚の画像に対して数千回CNNを回さなければならず、計算時間がかなりかかってしまっていた。そのため、リアルタイムの検出が難しかった。  

# (2) Fasterとあるが、どういった仕組みで高速化したのか

<Fasterについて>  
Faster R-CNNはRegionProposalもCNN化することで物体検出モデルを全てDNN化し、高速化している。  
これは、CNN出力（特徴マップ）を元にregion proposal(物体があるっぽい領域を抽出）するモデルを構築している。  
実際のregion proposalは3~4層ほどのCNNで構成可能で小さい。  
Regionproposalの軽量化に成功したため、大幅に高速化が実現した。

<これまでの技術>  
Fast R-CNNではRegion Proposalに従来技術であるSelective Searchを使用しており、そこが速度ボトルネックとなっていた。  
Selective Searchとはピクセルレベルで類似する領域をグルーピングしていくことで候補領域を選出するルゴリズム。  
要は似たような特徴を持つ領域を結合していき、１つのオブジェクトとして抽出している。  

<これまでのSelective Searchの具体的ステップ>  
※入力画像で物体らしさなどをあらかじめ評価しておき、領域候補数を絞り込む手法とまとめられる。
①画像を小さな領域群に分割
②隣合う領域同士の類似度を計算し、最も類似度の高い領域を統合
③①と②のステップを領域が１つになるまで繰り返す
つまり、階層的に得られた各画像領域をちょうど囲む短形領域を物体領域候補とする手法。

# (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

Two-stageの手法では第一ステージで物体を推定し、第二ステージで物体の認識（クラス分類）やbounding boxの座標の推定を行う二段階の作業を行なっている。
対して、One-stageの手法では、領域候補の推定と、クラスやbounding boxの座標の推定を1段階で行う。

# (4) RPNとは何か。

物体の領域座標を特定するために用いるネットワークである。  
畳み込み層の特徴マップの局所領域ごとに、物体らしさのスコアが付与された複数のバウンディングボックスをRPNが提示する。  
このスコア付きバウンディングボックスを提案するために、バウンディングボックスのパラメータを予測する回帰ネットワークと、  
物体の有無を予測する分類ネットワークを結合することでRPNは実現されている。

# (5) RoIプーリングとは何か。

画像からある程度畳み込み処理を行ったfeature mapから、region proposalにあたる部分領域をうまく「固定次元サイズのfeature map」として抽出するプーリングである。つまり、RoI Poolingは分類を行う層への入力を固定次元にする役割がある。物体検出の領域は非常に可変長であり、例えば、同じ画像から検出された椅子とりんごが同じサイズではないのと同じようなもの。その得られた領域を次の分類のネットワークで処理するため、固定次元に縮小する処理を行う。


<img src="./images/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f3133393830392f39616366363937312d363263332d383265312d656136312d3562613062623736393862312e706e67.png">



具体的にはまず、region proposalの座標を整数値に丸め、上記の赤い外接矩形を得ます。この時点で、最大0.5ピクセルのずれが発生します。これは、元画像ではCNNのstrideの半分に相当し、今回の例では32/2=16ピクセルのずれに相当します。

その後、その外接矩形をfeature mapのサイズと同じビン数（ここでは3x3）に等分割します。そして、feature mapのRoI内のピクセルを、それぞれ3x3つのビンのいずれかに割り当て、割り当てられたビンの中でmaxやaverageを取ることで、RoI Poolingの出力を得ます。

上図では、市松模様でfeature mapの各ピクセルの割り当てを表現しています。丸め方やピクセルの割り当て方は色々な方法が考えられます（切り捨ててしまうとか）が、上記の図のような形が誤差を小さくできるのではないでしょうか。
（こちらの動画では、RoIの取得時や、3x3の分割時に切り捨てを行う説明になっているので、かなり誤差があるように見えてしまいますが、オリジナルの論文でも"round"という表記をしています）

何れにせよ、RoIの取得と、ピクセルの割り当て時に丸め誤差が発生し、これがセグメンテーションなどの位置ずれがあまり許されないアプリケーションでは問題になります。

# (6) Anchorのサイズはどうするのが適切か。

本来は対象となる物体の一般的な縦横比率を用いてサイズを設定する必要がある。
PASCAL VOC 2007のデータセットにおいては、128^2, 256^2, 512^2でかつアスペクト比が1:1のデータセットであれば、  
mAPスコアは「3 scales　３ ratios」のデフォルト値に近い６9.8に達したので、こういったスケールで実施するのが良いと思われる。

# (7) 何というデータセットを使い、
# 先行研究に比べどういった指標値が得られているか。

VGG16を用いた中、RPN＋ Fast R-CNNのシステムを用いると

* PASCAL VOC2007や2012
* MS COCOのデータセット

においてSOTAの物体検出精度を出しながら、GPU上で5fpsの計算速度を実現した。  
尚、SS + Fast R-CNNのシステムを用いた場合には、0.5fpsを記録した。

また、

※ILSVRCやCOCOの2015年のコンペティションにおいて、Faster R-CNNはいくつかの部門において1stの位置を占めたエントリーの礎となった。

# (8) （アドバンス）Faster R-CNNよりも新しい物体検出の論文では、 
# Faster R-CNNがどう引用されているか。  